In [1]:
import torch 
import torch.nn as nn 
import torch.optim as optim 

In [2]:
from Subclassing_models import SubclassSequential, SubclassModel, SubclassFunctionalModel

In [3]:
n_neurons = 8
seq_model = SubclassSequential(n_neurons)
print(seq_model)

SubclassSequential(
  (model): Sequential(
    (hidden_linear): Linear(in_features=1, out_features=8, bias=True)
    (hidden_activation): Tanh()
    (output_linear): Linear(in_features=8, out_features=1, bias=True)
  )
)


In [4]:
subclass_model = SubclassModel()
print(subclass_model)

SubclassModel(
  (hidden_linear): Linear(in_features=1, out_features=13, bias=True)
  (hidden_activation): Tanh()
  (output_linear): Linear(in_features=13, out_features=1, bias=True)
)


In [5]:
subclassFunctional_model = SubclassFunctionalModel()
print(subclassFunctional_model)

SubclassFunctionalModel(
  (hidden_linear): Linear(in_features=1, out_features=14, bias=True)
  (output_linear): Linear(in_features=14, out_features=1, bias=True)
)


***

In [6]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c).unsqueeze(1) # <1>
t_u = torch.tensor(t_u).unsqueeze(1) # <1>


n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)
train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]


t_u_train = t_u[train_indices]
t_c_train = t_c[train_indices]

t_u_val = t_u[val_indices]
t_c_val = t_c[val_indices]

t_un_train = 0.1 * t_u_train
t_un_val = 0.1 * t_u_val

In [7]:
def training_loop(n_epochs:int , optimizer, model, loss_fn, t_u_train, t_u_val, t_c_train, t_c_val):
    
    # _Start: training loop for n_epochs
    for epoch in range(1, n_epochs + 1):
        
        # _Start: for training set 
        t_p_train = model(t_u_train) # <1>
        loss_train = loss_fn(t_p_train, t_c_train)
        # _End: for training set 
        
        # _Start: for validation set 
        with torch.no_grad():
            t_p_val = model(t_u_val) # <1>
            loss_val = loss_fn(t_p_val, t_c_val)
            assert loss_val.requires_grad == False
        # _End: for validation set 
            
         # _Start: backpropagation and update 
        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()
        # _End: backpropagation and update 

        if epoch == 1 or epoch % 1000 == 0:
            print('Epoch {}, Training loss {}, Validation loss {}'.format(
                epoch, float(loss_train), float(loss_val)))
    # _End: training loop for n_epochs

In [8]:
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

***

In [11]:
optimizer = optim.SGD( seq_model.parameters(),  lr=1e-3)

training_loop(
    n_epochs = 5000, 
    optimizer = optimizer,
    model = seq_model,
    loss_fn = loss_fn,
    t_u_train = t_un_train,
    t_u_val = t_un_val, 
    t_c_train = t_c_train,
    t_c_val = t_c_val)

print('output: ', seq_model(t_un_val))
print('answer: ', t_c_val)
print('hidden: ', seq_model.hidden_linear.weight.grad)

Epoch 1, Training loss 1.320799708366394, Validation loss 7.433529853820801
Epoch 1000, Training loss 1.3198957443237305, Validation loss 7.438112735748291
Epoch 2000, Training loss 1.3191473484039307, Validation loss 7.431526184082031
Epoch 3000, Training loss 1.3184970617294312, Validation loss 7.420357704162598
Epoch 4000, Training loss 1.3179190158843994, Validation loss 7.407679557800293
Epoch 5000, Training loss 1.3173913955688477, Validation loss 7.394316673278809
output tensor([[8.3193],
        [3.5675]], grad_fn=<AddmmBackward>)
answer tensor([[6.0000],
        [0.5000]])


AttributeError: 'SubclassSequential' object has no attribute 'hidden_linear'